In [3]:
from rdkit import Chem
from rdkit import rdBase
# Suppress stereochemistry warnings (optional)
rdBase.DisableLog('rdApp.warning')

input_sdf = 'chembl_34_sdf/chembl_34.sdf'
output_sdf = 'filtered_chembl_34.sdf'

writer = Chem.SDWriter(output_sdf)
for mol in Chem.ForwardSDMolSupplier(input_sdf):
    if mol is not None:
        writer.write(mol)
writer.close()

sdf_file = 'filtered_chembl_34.sdf'

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Chaitanya\ML Projects\QSAR Molecular Descriptor Prediction\.conda\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\chanu\AppData\Local\Temp\ipykernel_33080\1596949343.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Chaitanya\ML Projects\QSAR Molecular Descriptor Prediction\.conda\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Chaitanya\ML Projects\QSAR Molecular Descriptor Prediction\.conda\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Chaitanya\ML Projects

In [2]:
import pybel
from openbabel import pybel
import random
import time
import sys
import pandas as pd
from rdkit import Chem,rdBase


# Define the path to the SDF file
sdf_file = 'chembl_34_sdf/chembl_34.sdf'

# Define the total number of molecules to sample
total_samples = 100000

# Suppress stereochemistry warnings (optional)
rdBase.DisableLog('rdApp.warning')
# Function to process molecules in batches and sample randomly with manual progress bar
def process_molecules_random_sample(file_path, total_samples):
    mols = []
    total_mols = 0
    for mol in pybel.readfile("sdf", file_path):
        total_mols += 1

    sample_indices = set(random.sample(range(total_mols), total_samples))

    bar_length = 20
    start_time = time.time()

    for i, mol in enumerate(pybel.readfile("sdf", file_path)):
        if i in sample_indices:
            rdkit_mol = mol.to_rdkit()
            if rdkit_mol is not None:
                Chem.RemoveStereochemistry(rdkit_mol)
                try:
                    Chem.SanitizeMol(rdkit_mol)
                    mols.append(rdkit_mol)
                    progress = len(mols) / total_samples
                    filled_length = int(round(bar_length * progress))
                    bar = '█' * filled_length + '-' * (bar_length - filled_length)
                    elapsed_time = time.time() - start_time
                    print(f'\rSampling Molecules: |{bar}| {progress:.1%} [{elapsed_time:.1f}s]', end='')
                    sys.stdout.flush()
                except Exception as e:
                    print(f"\nError sanitizing molecule {i}: {e}")
            if len(mols) >= total_samples:
                break
    print("\n")
    return mols

sampled_mols = process_molecules_random_sample(sdf_file, total_samples)

# Convert Mol objects to SMILES strings
data = []
for mol in sampled_mols:
    smiles = Chem.MolToSmiles(mol)
    data.append(smiles)

# Create a DataFrame with SMILES strings
df_smiles = pd.DataFrame(data, columns=['SMILES'])

# Save the DataFrame to a CSV file
df_smiles.to_csv('sampled_100k_molecules_smiles.csv', index=False)


AttributeError: module 'pybel' has no attribute 'readfile'